inputs

In [3]:
input_gene_id = "ADA2" ## entrez ID
input_entrez_id = 51816
input_uniprot_id = "Q9NZK5"
input_protein_file = "ADA2.txt" ## isoform file

optional inputs

In [4]:
drosophila_entrez_id = 7227
homo_sapiens_entrez_id = 9606

script_folder = "oracle_scripts"
get_protein_info_script = f"{script_folder}/get_protein_info.py"
clustalw_script = f"{script_folder}/clustalw.sh"

dry_run = "yes"

imports

In [5]:
import oracle_functions
import requests
import pandas as pd
import os
import importlib

importlib.reload(oracle_functions)

<module 'oracle_functions' from 'l:\\Lab-Rusan\\Jacie\\00.code\\00.git\\oracle_scratch\\oracle_functions.py'>

code

orthologs and alignments

In [6]:
## make ortholog output folder
ortholog_and_alignment_output_folder = f"{input_gene_id}_ortholog_and_alignments_output"
os.system(f"mkdir {ortholog_and_alignment_output_folder}")

## getting and filtering DIOPT orthologs
diopt_results, diopt_file = oracle_functions.pull_diopt_orthologs(homo_sapiens_entrez_id, drosophila_entrez_id, input_entrez_id, ortholog_and_alignment_output_folder)
filtered_diopt_results, filtered_diopt_file = oracle_functions.filter_diopt_results(diopt_results, diopt_file, ortholog_and_alignment_output_folder)

## getting protein info for alignment
diopt_id_list = filtered_diopt_results["entrez_id"].to_list()
diopt_id_string = " ".join(map(str, diopt_id_list))
diopt_fasta = "protein_orthologs.fasta"
command = f"python {get_protein_info_script} {diopt_id_string} {ortholog_and_alignment_output_folder}/protein_orthologs.zip {diopt_fasta}"
os.system(command)

## combine into one file
combined_file = f"{ortholog_and_alignment_output_folder}/combined_proteins.fasta"
with open(input_protein_file, 'r') as f1:
    data1 = f1.read()
with open(diopt_fasta, 'r') as f2:
    data2 = f2.read()
with open(combined_file, 'w') as cf:
    cf.write(data1)
    cf.write("\n")
    cf.write(data2)

# submit clustalw script
if dry_run == "no":
    os.system(f"sbatch {clustalw_script} -INFILE={combined_file}")

c:\Users\chengj7\AppData\Local\anaconda3\envs\biogrid\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flyrnai.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found DIOPT orthologs


evolution: phylogenetic tree

In [7]:
alignment_file = f"{ortholog_and_alignment_output_folder}/msa_test.dnd"
oracle_functions.visualize_phylogenetic_tree(alignment_file, output_file=f"{ortholog_and_alignment_output_folder}/phylo_tree.png")

Tree saved to ADA2_ortholog_and_alignments_output/phylo_tree.png


<Figure size 1000x800 with 0 Axes>

mutation effects

In [8]:
url = "http://v1.marrvel.org/data/clinvar"
req = requests.get(url, params = {"geneSymbol": input_gene_id})
df = pd.read_json(req.text)

# Filter the DataFrame to include only rows where the title contains "(p."
filtered_df = df[df['title'].str.contains(r'\(p\.', na=False)]
filtered_df = filtered_df.reset_index(drop=True)

# Extract the string between parentheses that contains "p."
filtered_df['protein_change'] = filtered_df['title'].str.extract(r'\(([^)]*p\.[^)]*)\)')

# Remove the "p." prefix from the extracted protein change
filtered_df['protein_change'] = filtered_df['protein_change'].str.replace('p.', '', regex=False)

significance_description = []
for i, row in filtered_df.iterrows():
    desc = filtered_df["significance"][i]["description"]
    significance_description.append(desc)
filtered_df["significance_description"] = significance_description

amino_acid_position = []
for i in filtered_df["protein_change"].to_list():
    position = oracle_functions.extract_numbers(i)
    if len(position) > 1:
        raise ValueError("Something went wrong. There should not be more than one amino acid position")
    amino_acid_position.append(position[0])
filtered_df["amino_acid_position"] = amino_acid_position

# Generate PyMOL script
dict = oracle_functions.create_color_dict(filtered_df, 'amino_acid_position', 'significance_description')
filtered_df["color"] = filtered_df["amino_acid_position"].map(dict)
oracle_functions.generate_pymol_script_alleles(filtered_df, "amino_acid_position", "color", f"{input_gene_id}_color_alleles.pml")


In [9]:
url = f"https://rest.uniprot.org/uniprotkb/{input_uniprot_id}"
req = requests.get(url)
data = req.json()

temp_list = []
for i in range(0, len(data["features"])):
    if data["features"][i]["type"] == "Active site" or data["features"][i]["type"] == "Binding site":
        temp_list.append(data["features"][i])
df = pd.DataFrame(temp_list)
df_flattened = pd.json_normalize(temp_list)
oracle_functions.generate_pymol_script_domains(df_flattened, f"{input_gene_id}_color_domains.pml")

PyMOL script saved to ADA2_color_domains.pml


In [10]:
data

{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
 'primaryAccession': 'Q9NZK5',
 'secondaryAccessions': ['A8K9H4', 'Q6ICF1', 'Q86UB6', 'Q8NCJ2', 'Q96K41'],
 'uniProtkbId': 'ADA2_HUMAN',
 'entryAudit': {'firstPublicDate': '2002-04-16',
  'lastAnnotationUpdateDate': '2024-11-27',
  'lastSequenceUpdateDate': '2007-01-09',
  'entryVersion': 180,
  'sequenceVersion': 2},
 'annotationScore': 5.0,
 'organism': {'scientificName': 'Homo sapiens',
  'commonName': 'Human',
  'taxonId': 9606,
  'lineage': ['Eukaryota',
   'Metazoa',
   'Chordata',
   'Craniata',
   'Vertebrata',
   'Euteleostomi',
   'Mammalia',
   'Eutheria',
   'Euarchontoglires',
   'Primates',
   'Haplorrhini',
   'Catarrhini',
   'Hominidae',
   'Homo']},
 'proteinExistence': '1: Evidence at protein level',
 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000312',
      'source': 'HGNC',
      'id': 'HGNC:1839'}],
    'value': 'Adenosine deaminase 2'},
   'ecNumbers': [{'evidences'

rna seq expression patterns

In [11]:
## uses previous uniprot api call

go_list = []

for i in range(0,len(data["uniProtKBCrossReferences"])):
    database_type = data["uniProtKBCrossReferences"][i]["database"]
    if database_type == "GO":
        go_list.append(data["uniProtKBCrossReferences"][i]["properties"][0]["value"])

function_dict = {'C': 'Cellular Component',
        'F': 'Molecular Function',
        'P': 'Biological Process'}

# Prepare lists for DataFrame columns
# gene_names = []
function_list = []
purpose_list = []
mapped_values = []

# Process each item in the list
for item in go_list:
    # Split the item on the colon
    parts = item.split(':')
    
    # Append to respective lists

    function_list.append(parts[1])
    mapped_values.append(function_dict.get(parts[0], "Unknown"))  # Use "Unknown" if not found in the dictionary

# Create a DataFrame
df = pd.DataFrame({
    "gene_name": input_gene_id,
    "second_element": function_list,
    "mapped_value": mapped_values
})


df.to_csv(f"{input_gene_id}_related_GO_terms.csv")

In [12]:
data

{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
 'primaryAccession': 'Q9NZK5',
 'secondaryAccessions': ['A8K9H4', 'Q6ICF1', 'Q86UB6', 'Q8NCJ2', 'Q96K41'],
 'uniProtkbId': 'ADA2_HUMAN',
 'entryAudit': {'firstPublicDate': '2002-04-16',
  'lastAnnotationUpdateDate': '2024-11-27',
  'lastSequenceUpdateDate': '2007-01-09',
  'entryVersion': 180,
  'sequenceVersion': 2},
 'annotationScore': 5.0,
 'organism': {'scientificName': 'Homo sapiens',
  'commonName': 'Human',
  'taxonId': 9606,
  'lineage': ['Eukaryota',
   'Metazoa',
   'Chordata',
   'Craniata',
   'Vertebrata',
   'Euteleostomi',
   'Mammalia',
   'Eutheria',
   'Euarchontoglires',
   'Primates',
   'Haplorrhini',
   'Catarrhini',
   'Hominidae',
   'Homo']},
 'proteinExistence': '1: Evidence at protein level',
 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000312',
      'source': 'HGNC',
      'id': 'HGNC:1839'}],
    'value': 'Adenosine deaminase 2'},
   'ecNumbers': [{'evidences'